In [ ]:
import stata_setup
stata_setup.config("C:/Program Files/Stata17/", "mp")

## Ridge Regression

The ridge regression estimator

$$
\widehat{\beta}_{\text {ridge }}=\left(\boldsymbol{X}^{\prime} \boldsymbol{X}+\lambda \boldsymbol{I}_{p}\right)^{-1} \boldsymbol{X}^{\prime} \boldsymbol{Y}
$$

where $\lambda>0$ is called the _ridge_ parameter.

In [ ]:
%%stata -eret steret
use ../Data/breathe, clear
quietly do ../Do/no2
display "$cc"
display "$fc"
quietly elasticnet linear react no2_class $cc i.($fc), alpha(0) lambda(5.34) nolog

In [ ]:
steret['e(b)']

In [ ]:
%%stata
elasticnet linear react no2_class $cc i.($fc), alpha(0) lambda(0.1(.005)0.3) folds(1036) nolog

In [ ]:
%%stata
use ../Data/breathe, clear
quietly do ../Do/no2
display "$cc"
display "$fc"
quietly elasticnet linear react no2_class $cc i.($fc), alpha(0) lambda(0.225) nolog
matrix list r(table)

In [ ]:
%%stata
predict reac_hat, xb
gen ehat = react - reac_hat
set scheme s1mono
histogram ehat